# Analítica de texto

> _Preparado por: [Juan Javier Santos Ochoa](https://twitter.com/jjsantoso) ([LNPP](https://www.lnpp.mx/))_

* En este tutorial veremos cómo hacer algunas tareas de analítica de texto usando métodos de aprendizaje no supervizado. La idea es obtener algo de conocimiento sobre estos texto sin tener necesidad que revisarlos extensivamente.

* Las tareas más comunes dentro del análisis de texto incluyen categorización, clusterización, extracción de entidades, análisis de sentimiento, resumen de documentos, predicción de palabras y [generación automática de contenido](https://automatedinsights.com/)

* Algunas de las aplicaciones incluyen, por ejemplo, calcular la [polaridad/sentimiento de tweets](https://www.inegi.org.mx/app/animotuitero/#/app/multiline), identificar el idioma de una publicación, detectar si una noticia menciona nombres de personas, lugares u otras características, identificar los tópicos de los que tratan un conjunto de documentos o [agrupar programas sociales similares](https://github.com/plataformapreventiva/social_programs_text_analysis) según la descripción de sus reglas de operación.

* A un conjunto de datos para hacer análisis de texto se le llama corpus y cada texto individual se conoce como un documento.

* Los modelos de ML no pueden recibir texto directamente, sino que cada documento se debe procesar de manera que tenga una representación vectorial numérica que pueda ser entendida por los algoritmos.

* En esta sesión veremos cómo procesar un corpus y usar algoritmos usuales de ML para hacer algunas de las tareas comunes en analítica de texto.
* Primero veremos algunas tareas comunes de preprocesamiento de texto y luego veremos algunas formas para representar un texto de forma vectorial.
* Esta sesión está basada principalmente en el libro de Müller, A. C., & Guido, S. (2016). Introduction to machine learning with Python: a guide for data scientists. " O'Reilly Media, Inc.".

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, strip_accents_ascii
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import pairwise_distances
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from string import punctuation

import matplotlib.pyplot as plt
import matplotlib._color_data as mcd # paleta de colores
from matplotlib import markers

print(sklearn.__name__, sklearn.__version__)
print(np.__name__, np.__version__)
print(pd.__name__, pd.__version__)
%matplotlib inline

# Discursos
* Tomaremos los discursos de la Junta de Gobierno del Banco de México: https://www.banxico.org.mx/publicaciones-y-prensa/discursos/discursos-junta-gobierno-pala.html
* Se omitieron los discursos en inglés.
* Cada texto está limitado a las primeras 5000 palabras.

In [ ]:
discursos = pd.read_csv('datos/discursos.csv')
discursos.head()

# Preprocesamiento

* Antes de entrenar cualquier modelo tenemos que preprocesar el texto para homogeneizarlo y remover algunas palabras o caracteres que pueden ser irrelevantes.

## Minúsculas y remover acentos
* Como nuestro texto está en un dataframe de pandas podemos usar el método `str.lower()` para convertir todo a minúsculas.
* También podemos usar el método `.apply()` para aplicar una función personalizada. sklearn tiene la función `strip_accents_ascii`, dentro del módulo `sklearn.feature_extraction.text`, para remover acentos.

In [ ]:
discursos['texto'] = discursos['texto'].str.lower().apply(strip_accents_ascii)
discursos['texto'].head()

## Stopwords

* Las stopwords son palabras muy comunes que usualmente no aportan mucha información. Estas palabras suelen ser artículos o preposiciones.
* En español podemos usar la lista de stopwords que viene en la librería [`nltk`](https://www.nltk.org/), dentro del módulo `stopwords`. La libreríaa ya viene instalada en Anaconda.
* Dependiendo de la aplicación. podemos agregar nuestras propias stopwords.
* A continuación vamos a usar las stopwords de nltk que están guardadas en el archivo `datos/stopwords_spanol.csv` y creamos una función que las remueve de cualquier texto.

In [ ]:
stops = pd.read_csv('datos/stopwords_spanol.csv')['stopwords'].tolist()
def remueve_stopwords(texto):
    p = [t for t in texto.lower().split() if t not in stops]
    return ' '.join(p)

In [ ]:
discursos['texto'] = discursos['texto'].apply(remueve_stopwords)
discursos['texto'].head()

## Remover puntuación y números

* También puede ser útil remover signos de puntucación y números
* Dentro de la librería `string` se encuentra la variable `punctuation` que contiene los principales signos de puntuación. En español debemos agregarle algunos signos de apertura (¡¿)
* Usamos el método `.str.replace()` de las series de Pandas.

In [ ]:
discursos['texto'] = discursos['texto'].str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿\d]', '')
discursos['texto'].head()

# Bolsa de palabras (Bag of Words)

* BoW es una de las formas más simples de representar texto y usualmente se desempeña bastante bien en ciertas tareas, como categorización.
* Consiste en contar la ocurrencia de todas las palabras en cada documento.
* En BoW no importa la estructura gramatical del texto (oraciones, párrafos) ni el orden de las palabras, simplemente importa cuántas veces ocurren.
* Para hacer una representación BoW de un corpus se necesitan tres pasos:
    1. Tokenización: consiste en descomponer el texto en elementos más pequeños, por ejemplo oraciones o palabras.
    2. Vocabulario: Se recolectan todas las palabras que aparecen en todos los documentos y se asigna un identificador numérico único a cada una, usualmente en orden alfabético.
    3. Codificación: Para cada documento se cuenta la frecuencia de todas las palabras del vocabulario.

* Con esta información se construye una matriz en la que cada fila representa un documento y las columnas una palabra del vocabulario:

<img src="imagenes/bow_matriz.PNG" width=600>


* Veamos el siguiente ejemplo con solo 3 documentos:

```python
["Me gusta el café sin azucar y sin crema", "Desayuno café con leche y pan con fruta", "Solo tomo leche deslactosada"]
```

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>azucar</th>      <th>café</th>      <th>con</th>      <th>crema</th>      <th>desayuno</th>      <th>deslactosada</th>      <th>el</th>      <th>fruta</th>      <th>gusta</th>      <th>leche</th>      <th>me</th>      <th>pan</th>      <th>sin</th>      <th>solo</th>      <th>tomo</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>1</td>      <td>1</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>2</td>      <td>0</td>      <td>0</td>    </tr>    <tr>      <th>1</th>      <td>0</td>      <td>1</td>      <td>2</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>      <td>0</td>    </tr>    <tr>      <th>2</th>      <td>0</td>      <td>0</td>      <td>0</td>      <td>0</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>      <td>0</td>      <td>1</td>      <td>0</td>      <td>0</td>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>

* Tenemos un vocabulario de 15 palabras.
* Con esta representación ahora podemos usar algunos de los algoritmos de ML que hemos visto.

## Implementación de BOW

* En sklearn está la función `CountVectorizer()` dentro del módulo `sklearn.feature_extraction.text` que nos genera una matriz con el conteo de cada palabra para cada documento.

In [ ]:
frases = ["Me gusta el café sin azucar y sin crema", "Desayuno café con leche y pan con fruta", "Solo tomo leche deslactosada"]
vect = CountVectorizer()
vect.fit_transform(frases).toarray()

* Podemos ver el vocabulario con el atributo `.vocabulary_`. Por defecto, todas las palabras son convertidas a minúsculas.
* El vocabulario es un diccionario en el que el _key_ es cada palabra y el _value_ es un índice numérico basado en el orden.

In [ ]:
vect.vocabulary_

## Tf-idf

* Obtuvimos buenos resultados con el conteo de palabras, sin embargo, en general ocurrirá que en documentos más largos habrá un mayor conteo que en documentos más cortos, aunque pertenezcan a la misma categoría.

* Una forma de controlar esto es dividiendo el número ocurrencias de cada palabra sobre el número total de palabras en el documento. Esto se conoce como term-frecuency, abreviado como tf.

* Un problema de contar las frecuencias, ya sea en términos absolutos o relativos, es que muchas palabras comunes aparecen en muchos documentos, por tanto no son tan informativas. En cambio, hay otras palabras que aparecen con menor frecuencia en ciertos documentos pero que son más distintivas y por tanto aportan más información al modelo. Entonces, una forma de mejorar las predicciones puede ser modificando el peso de cada palabra de forma inversamente proporcional a su frecuencia en el corpus. Esto se conoce como inverse document frecuency, idf.

* Al aplicar ambas trasnformaciones obtenemos tf-idf: “Term Frequency times Inverse Document Frequency”. La intuición es que al aplicar este método se le da más peso a las palabras que ocurren frecuentemente en ciertos documentos, pero no en muchos.

* En sklearn podemos usar `TfidfVectorizer`. sklearn usa en particular la fórmula:

$$\text{tfidf}(w, d) = \text{tf}\log{(\frac{N+1}{N_w +1})} + 1$$


 Donde: $N$ es el número de documentos en los datos de entrenamiento, N_w es el número de documentos en los datos de entrenamiento en los que aparece la palabra $w$ y tf es el número de veces que aparece la palabra $w$ en el documento $d$
 
 * Existe una opción `sublinear_tf` que si es activada (True) reemplaza tf por $1+\log (\text{tf})$

In [ ]:
tf_transf = TfidfVectorizer()
tf_transf.fit_transform(frases).toarray()

## N-gram vectorizer

* Los N-gramas son combinaciones de N palabras contiguas juntas. Por ejemplo, en la frase "A mí me gusta el café" tenemos los siguientes bigramas:
```python
("a", "mí"), ("mí", "me"), ("me", "gusta"), ("gusta", "el"), ("el", "café")
```

* Usualmente la unión de dos o tres palabras es más informativa que una sola palabra, por lo que podemos obtener mejores resultados en el caso que existan palabras con ambiguedad, lo que suele ser muy común en el análisis de sentimientos.

* Tanto `CountVectorizer()` como `TfidfVectorizer()` tienen un parámetro `ngram_range` que debe ser una tupla especificando el número mínimo y máximo de N-gramas que se adicionarán. Por ejemplo, ngram_range=(1, 3) indica que se van a adicionar las palabras solas y todas las combinaciones de 2 y 3 palabras contiguas.

* Una dificultad de usar N-gramas es que es más difícil encontrar la combinación de N-palabras contiguas. Por tanto no se suele especificar más allá de trigramas y se usa cuando se tiene un corpus suficientemente grande.

In [ ]:
ngram_vectorizer = CountVectorizer(ngram_range=(2, 2))
pd.DataFrame(data=ngram_vectorizer.fit_transform(frases).toarray(), columns=ngram_vectorizer.get_feature_names())

## Limitaciones de BoW y Tf-IDF
* BoW no captura información secuencial. Documentos con diferentes significados pero iguales palabras tienen la misma representación:
<img src="imagenes/bow_limitacion.PNG" width=600>
* No captura información semántica. Palabras que se escriben igual pero tienen diferentes significados se capturan igual. Por ejemplo, banco puede ser una institución o una silla.
* Para superar estas dificultades es necesario usar modelos que puedan captar de mejor manera el caracter secuencial del texto y aprovechar el contexto para evitar ambiguedades. Algunos de estos modelos muy populares son [Word2vec](https://www.tensorflow.org/tutorials/text/word_embeddings) y las [Redes Neuronales Recurrentes](https://www.tensorflow.org/tutorials/text/text_classification_rnn) (RNN), pero están más allá del objetivo de este tutorial. 
* A continuación usaremos el Análisis semántico latente para intentar obtener representaciones que comprenden un poco mejor el contenido de un texto.

# Análisis semántico latente (LSA)

* El análisis semántico latente es una técnica para reducir la dimensionalidad de una matriz de frecuencia de términos. Usa la técnica de descompsición de valores singulares, parecido a PCA, pero a diferencia de esta, LSA funciona mejor con datos esparsos y los datos no necesitan ser centrados alrededor de la media.
* LSA permite representas cada documento en una dimensión mucho menor y además el resultado tiende a encontrar parecidos semánticos en los documentos, de tal manera que documentos que tratan sobre los mismos temas, aunque no usen las mismas palabras, tienden a tener representaciones más parecidas.

* Suponiendo que tenemos una matriz $X$ en la que cada elemento $(i, j)$ describe la ocurrencia del término $j$ en el documento $i$. Con $m$ documentos y un vocabulario de tamaño $n$.

$$
X=\begin{pmatrix}
x_{1, 1} & \cdots & & x_{1, n} \\
\vdots & \vdots & & \vdots \\
x_{i, 1} & x_{i, j} & & x_{i, n} \\
\vdots & \vdots & & \vdots \\
x_{m, 1} & \cdots & & x_{m, n} \\
\end{pmatrix} 
$$

* Por la teoría de algebra lineal, existe una descomposición de valores singulares de la matrix $X$:

    $$X = U\Sigma V^T$$

    Donde $U$ y $V$ son matrices ortogonales ($Q^TQ = QQ^T = I$) y $\Sigma$ es una matriz diagonal. $U$ y $V$ contienen los llamados vectores propios izquierdos y derechos, respectivamente y los valores de la diagonal $\Sigma$ se conocen como los valores propios.
    
* Si seleccionamos los $k$ valores propios más altos $\Sigma_k$, junto con sus respectivos vectores propios $U_k$ y $V_k$, entonces obtenemos la aproximación de rango $k$ de $X$ con mínimo error:

$$ X_k = U_k \Sigma_k V^T_k$$

* Esta aproximación tiene tamaño $m\times k$, con $k\leq n$.
* Si queremos representar un nuevo documento $d_i$ de la forma reducida entonces aplicamos la operación:
 $$\hat{d_i} = \Sigma^{-1}_kV^T_k d_i$$
 
* La representación del corpus en un espacio de menor dimensión usando descomposición de valores singulares genera lo que se conoce como un "espacio semántico".
* Con el espacio de menor dimensión es posible desarrolar algunas tareas como:
    * Clasificar.
    * Comparar la similitud de documentos.
    * Clusterizar.
    * Encontrar palabras relacionadas.
    


## Truncated SVD

* En sklearn podemos hacer LSA aplicando la función `TruncatedSVD()` al resultado de una matriz de frecuencia de terminos `CountVectorizer()` o `TfidfVectorizer()`
* Antes debemos preprocesar el texto.
* En la función `TruncatedSVD()` debemos especificar cuál es el número de componentes al que queremos reducir la información de los documentos. Este número debe ser menor o igual al total de documentos.

* El método `.fit_tranform()` ajusta el modelo y genera una matriz de tamaño $(n\_documentos, n\_componentes )$

In [ ]:
svd_transformer = Pipeline([
                            ('tfidf', TfidfVectorizer(sublinear_tf=True)), 
                            ('svd', TruncatedSVD(n_components=100, n_iter=10, random_state=42))])

svd_matrix = svd_transformer.fit_transform(discursos['texto'])

In [ ]:
svd_matrix

## Similitud de documentos

* Como cada documento está representado por un vector, podemos considerar la distancia entre los vectores como una medida de la similaridad de los documentos.
* En el análisis de texto es usual usar una métrica conocidad como distancia coseno, $D_c$, que se deriva de la similitud coseno (cosine similarity).

$$D_c(A, B) = 1 - S_c(A, B)$$
$$S_c = \frac{\mathbf{A}\cdot\mathbf{B}}{||\mathbf{A}||\text{ }||\mathbf{B}||}$$

* En la gráfica anterior, la línea roja representa la distancia euclidiana, mientras que la azul representa la distancia coseno, esta última captura la dirección y es independiente de la magnitud de los vectores.
* En sklearn usamos la función `pairwise_distances` dentro del módulo `sklearn.metrics` para calcular una matriz de distancias, en particular especificamos la distancia 'cosine'.
* Le pasamos la matriz que contiene la representación vectorial de cada documento.
* La matriz que resulta tiene todas las combinaciones de distancias de los documentos: La i-ésima fila de la matriz contiene la distancia del i-ésimo documento con todos los demás.
* Para un documento, nos interesa encontrar los demás documentos con los que tienen menor distancia. 

In [ ]:
distance_matrix = pairwise_distances(svd_matrix, svd_matrix, metric='cosine')
distance_matrix.shape

In [ ]:
pd.options.display.max_colwidth = 300
index_query = 50
discursos.loc[[index_query], 'titulo']

* Hacemos una copia del DataFrame de los discursos y agregamos una columna con la distancia. De esta manera podemos ver cuál es el título y el texto de los documentos más parecidos.

In [ ]:
df_dist = discursos.copy()
df_dist['dist'] = distance_matrix[index_query]
df_dist.sort_values('dist').head()[['titulo', 'texto']]

## Clusterización de documentos

* Podemos usar la representación vectorial de los documentos como input para algún algoritmo de clusterización, como el de KMeans.
* Podemos probar varios valores de n_clusters y verificar usando el método del codo o el coeficiente de silueta. Sin embargo, tratándose de texto, siempre es bueno revisar que los clústeres tinen sentido.

In [ ]:
svd_cluster = Pipeline([
    ('svd_transform', svd_transformer),
    ('kmeans', KMeans(n_clusters=5) )
])

svd_cluster.fit(discursos['texto'])

* A continuación, hacemos una copia del DataFrame original y creamos una nueva columna que contenga el cluster al cual se asignó cada texto.

In [ ]:
discursos_cluster = discursos.copy()
discursos_cluster['grupo'] = svd_cluster.predict(discursos['texto'])
discursos_cluster.head(3)

In [ ]:
discursos_cluster['grupo'].value_counts().plot.bar(title='Distribución de documentos por grupo')

* A continuación revisamos los primeros 5 títulos de cada clúster.

In [ ]:
for g in discursos_cluster['grupo'].sort_values().unique():
    print('\n####### Grupo:', g, '#'*50)
    for t in discursos_cluster.query('grupo==@g')['titulo'].head():
        print(t.strip())

## Visualización del espacio de documentos

* Puede ser muy útil ubicar todos los documentos, o una parte de ellos, en un plano y ver cómo se distribuyen los clústeres y qué tan cercanos están unos documentos a otros.
* Sin embargo, cada documento está representado en un espacio de dimensión 100, lo que es imposible de visualizar directamente.
* Para ello, existe una técnica de reducción de la dimensionalidad llamada tSNE específicamente diseñada para poder visualizar en un plano de 2 o 3 dimensiones.
* En palabras generales, t-distributed Stochastic Neighbor Embedding (TSNE) intenta encontrar regiones "densas" de puntos en un espacio de alta dimensión y trata de hacer una representación en un plano 2D (o 3D) de tal manera que las diferentes regiones queden lo más alejadas posible entre ellas y los elementos de cada región lo más cerca entre ellos. La idea es preservar a los que eran los vecinos más cercanos en el plano de mayor dimensión en el espacio de menor dimensión.
* Aquí pueden encontrar una explicación completa de sus autores: https://www.youtube.com/watch?v=RJVL80Gg3lA
* En sklearn podemos hacer TSNE con la función `TSNE` dentro del módulo `sklearn.manifold`. A esta función le podemos ajustar los parámetros:
    * `perplexity`: es un número entero, se recomienda tome un valor de entre 5 y 50. Se ajusta visualmente viendo cuál genera una mejor separación.
    * `metric`: La función que mide la distancia entre los puntos.
    * `n_components`: el número de dimensiones al que se reducirán los puntos.
* Es importante homogenizar la escala de los datos antes de usar TSNE.

In [ ]:
X_embedded = TSNE(n_components=2, perplexity=50, metric='cosine').fit_transform(svd_matrix)
X_embedded.shape

* Hacemos una copia de los datos e introducimos dos nuevas columnas, que continen los valores obtenidos del TSNE

In [ ]:
discursos_tsne = discursos_cluster.join(pd.DataFrame(X_embedded, columns=['tsne_0', 'tsne_1']))
discursos_tsne.head(3)

* A continuación se grafican todos los documentos en un plano de 2 dimensiones. El color lo asignamos de acuerdo al clúster en el que lo clasificamos en el paso anterior. 
* Intuitivamente, los documentos que tratan los mismos temas deben estar más cerca unos de otros.
* El número de cada punto es el índice del documento.

In [ ]:
paleta = list(mcd.XKCD_COLORS.values())[::25] # paleta de colores
marcadores = list(markers.MarkerStyle.markers.keys())[1:] # Distintos marcadores
ax = plt.subplot()
for g in discursos_tsne['grupo'].unique():
    discursos_tsne.query('grupo==@g').plot.scatter(x='tsne_0', y='tsne_1', color=paleta[g], marker=marcadores[g], figsize=(10, 8), label=g, ax=ax)

for i, row in discursos_tsne.iterrows():
    ax.annotate(i, (row.tsne_0, row.tsne_1), fontsize=8)

ax.legend(title='grupos')

# Detección de tópicos con Latent Dirichlet Allocation

* Otra tarea muy popular que pueden hacer lo algoritmos de ML con datos de texto es la detección de tópicos. Esta consiste en encontrar tópicos o temas de los que hablan los documentos a partir de su contenido.
* Esta es una tarea de aprendizaje no supervisado, por lo que no se requiere de datos de entrenamiento con documentos que ya tengan un tópico asignado, sino que el algoritmo tratará de deducir cuáles son los tópicos.
* Nosotros debemos epecificar el número tópicos que queremos extraer.
* El algoritmo más usado para esta tarea es el de Latent Dirichlet Allocation (LDA). Este considera cada documento como una composición de varios tópicos, y cada documento es una colección de palabras que también una probabilidad de pertenecer a cada tópico. Los detalles del método están más allá del alacance de este tutorial.
* Un tópico es una colección de palabras. Esperaríamos que las palabras asociadas con un tópico tengan están relacionadas y tengan sentido juntas, sin embargo eso no está garantizado. 
* En sklearn podemos usar LDA con la función `LatentDirichletAllocation` del módulo `sklearn.decomposition`. Especificamos el número de tópicos con la opción `n_componentes`.
* Es buena idea limitar el número de palabras para evitar que términos irrelevantes determinen un tópico. En el ejemplo de abajo nos limitamos a 2000 palabras y exluimos a quellas que aparecen en más del 20% de los documentos.
* Con la librería[PyLDAvis](https://pyldavis.readthedocs.io/en/latest/readme.html) se pueden crear visualizaciones interactivas de los modelos LDA.

In [ ]:
lda = Pipeline([
    ('vect', CountVectorizer(max_features=2000, max_df=0.2)),
    ('lda', LatentDirichletAllocation(n_components=10, max_iter=25, random_state=42))
])
lda.fit(discursos['texto'])

* El atributo `.components_` contiene un puntaje para cada palabra y cada tópico. 

In [ ]:
lda['lda'].components_.shape

* Creamos un dataframe con el vocabulario y le adjuntamos a cada palabra el púntaje por tópico

In [ ]:
vocab_topic = pd.DataFrame(data=lda['vect'].vocabulary_.items(), columns=['palabra', 'indice'])\
    .sort_values('indice')\
    .set_index('indice')
vocab_topic.head()

In [ ]:
componentes = pd.DataFrame(lda['lda'].components_.T, columns=[f'topico_{i}' for i in range(lda['lda'].components_.shape[0])])
componentes.head()

In [ ]:
df_topicos = vocab_topic.join(componentes)
df_topicos.head()

* Con el DataFrame resultante podemos saber cómo se compone cada tópico. Las palabras más representativas de un tópico son aquellas que tienen un mayor valor a lo largo de la columna.
* Veamos las 20 palabras más representativas de cada tópico.

In [ ]:
for top in df_topicos.filter(like='topico').columns:
    palabras = df_topicos.sort_values(top).tail(20)[['palabra', top]]
    palabras.plot.barh(x='palabra', y=top, title=top)

* Para ver a qué tópico pertenece cada documento podemos usar el método `.transform()` sobre los documentos.
* El resultado es una matriz que contiene la probabilidad de que cada documento pertenezca a cada uno de los tópicos.
* Un documento se asigna al tópico con mayor probabilidad.

In [ ]:
doc_topic = pd.DataFrame(lda.transform(discursos['texto']))
doc_topic.head()

* A continuación hacemos una copia del DataFrame original y creamos una nueva columna que contenga el tópico asignado a cada documento.

In [ ]:
topico_documento = discursos.copy()
topico_documento['topico'] = doc_topic.idxmax(axis=1)
topico_documento.head()

* Veamos la distribución de los tópicos 

In [ ]:
topico_documento['topico'].value_counts().plot.bar(title='Distribución de documentos por tópico')